# Prepare input for ligand-receptor analysis

This notebook uses edge-edge distance meaurements between leukemia cells and every other cell to group cells into "close" (0-5 microns) and "far" (30+ microns) groups (with respect to closest leukemia cell).

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import math
import seaborn as sns

In [3]:
pred_location = '/data/Zhaolab/1_AMLCosMx/Final_scripts/4_CellTyping/4_CelltypesByTranscript/FromGege/celltype_v8-EP.csv'

In [4]:
dist_dir = '/data/Zhaolab/1_AMLCosMx/Final_scripts/5_SpatialAnalysis/1_MeasureCellDists/cell-cell_distances/'

In [5]:
# create dictionary of patient names
name_dirs = {'p1':'P51', 'p2':'P52', 'p3':'P53', 'p4':'P56', 'p5':'P57', 'p6':'P58'}
toGegeID = {'P51':'p1', 'P52':'p2', 'P53':'p3', 'P56':'p4', 'P57':'p5', 'P58':'p6'}

In [6]:
# read in celltype predictions
df = pd.read_csv(pred_location)

# reformat columns
df3 = pd.concat([df['x'], df['Unnamed: 0'].str.split('_', expand=True)], axis=1)
df3 = df3.rename(columns={"x": "celltype_detail", 0: "PtID", 3: "cell_id"})
df3['FOV'] = df3[1].apply(lambda x: x[-2:])
df3['patients'] = df3['PtID'].apply(lambda x: toGegeID[x])
df3 = df3.drop(columns=[1,2])
morph_predicted = df3[['patients', 'celltype_detail', 'FOV', 'cell_id']]

# define types to remove before further analysis
types_to_remove = ['NoCellAssigned','RBC','SmallCell','Unknown']

# filter out NA cells
morph_predicted = morph_predicted[~morph_predicted['celltype_detail'].isin(types_to_remove)]

# load FOV metadata to add timepoint info
FOV_metadata = pd.read_csv('/data/Zhaolab/1_AMLCosMx/Final_scripts/4_CellTyping/FOV_metadata.csv', index_col=0)
FOV_metadata.loc['P57_FOV17'] = ['C', 'CR'] # add row for P57 FOV17

morph_predicted['Timepoint'] = ''
for i in range(morph_predicted.shape[0]):
    alt_pt_name = name_dirs[morph_predicted['patients'].iloc[i]]
    fov = morph_predicted['FOV'].iloc[i]
    sample = alt_pt_name + '_FOV' + fov
    morph_predicted['Timepoint'].iloc[i] = FOV_metadata['Timepoint'].loc[sample]
    
morph_predicted

,patients,celltype_detail,FOV,cell_id,Timepoint
1,p1,CD8TeffectorGZMK,01,1,A
2,p1,MonocytesCD16,01,2,A
3,p1,Mega,01,3,A
4,p1,Mega,01,4,A
5,p1,NK,01,5,A
...,...,...,...,...,...
625140,p6,ErythroidProgenitors,23,7962,C
625141,p6,MatureB,23,7963,C
625142,p6,ErythroidProgenitors,23,7966,C
625143,p6,Mega,23,7971,C


In [8]:
# get cell type names
class_column = 'celltype_detail'
cluster_labels = list(set(morph_predicted[class_column].tolist()))
cluster_labels.sort()

# read colors from csv
colors = pd.read_csv('/data/Zhaolab/1_AMLCosMx/Final_scripts/4_CellTyping/4_CelltypesByTranscript/color_v8.csv', index_col=0)
colors = colors.T
cluster_colors = {}
for celltype in cluster_labels:
    cluster_colors[celltype] = [colors['blue'].loc[celltype], colors['green'].loc[celltype], colors['red'].loc[celltype]]
    
cell_types = list(cluster_colors.keys())

In [13]:
patients = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6']

In [14]:
All_DFs = []

for patient in patients:

    # subset to one patient
    one_pt = morph_predicted[morph_predicted['patients'] == patient]

    # get FOV names
    fovs = list(set(one_pt['FOV'].tolist()))

    for fov in fovs:

        # subset to one FOV
        one_fov = one_pt[one_pt['FOV'] == fov]

        # list of Leukemia cell IDs
        Leukemia_cell_ids = one_fov[one_fov['celltype_detail'] == 'LeukemiaCell']['cell_id'].astype(int).tolist()

        # list of non-leukemia cell IDs
        one_fov_noLK = one_fov[one_fov['celltype_detail'] != 'LeukemiaCell']
        other_ids = one_fov_noLK['cell_id'].astype(int).tolist()

        # read in distances computed between all cells
        min_dists = pd.read_csv(dist_dir + name_dirs[patient] + '_FOV' + fov + '_cell_contacts.csv', index_col=0)
        min_dists.columns = min_dists.columns.map(int)
        min_dists = min_dists.filter(items=other_ids, axis=0)
        min_dists = min_dists[Leukemia_cell_ids]
        min_dists = min_dists * 0.18 # convert from pixel distance to um distance

        # get min distance to closest LK cell
        min_LK_dists = min_dists.min(axis=1).to_frame(name='minLKdist_um')

        # concatenate mist dist col to rest of data
        one_fov_noLK['cell_id'] = one_fov_noLK['cell_id'].astype(int)
        one_fov_noLK_dists = one_fov_noLK.merge(min_LK_dists, how='left', left_on='cell_id', right_index=True)

        # place in distance from LK categories
        one_fov_noLK_dists['group'] = ''
        one_fov_noLK_dists['group'].loc[one_fov_noLK_dists['minLKdist_um'] >= 30] = '30+ um'
        one_fov_noLK_dists['group'].loc[one_fov_noLK_dists['minLKdist_um'] <= 5] = '0-5 um'

        All_DFs.append(one_fov_noLK_dists)
        del one_fov_noLK_dists

/tmp/ipykernel_632180/2252101337.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK['cell_id'] = one_fov_noLK['cell_id'].astype(int)
/tmp/ipykernel_632180/2252101337.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK_dists['group'].loc[one_fov_noLK_dists['minLKdist_um'] >= 30] = '30+ um'
/tmp/ipykernel_632180/2252101337.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

/tmp/ipykernel_632180/2252101337.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK['cell_id'] = one_fov_noLK['cell_id'].astype(int)
/tmp/ipykernel_632180/2252101337.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK_dists['group'].loc[one_fov_noLK_dists['minLKdist_um'] >= 30] = '30+ um'
/tmp/ipykernel_632180/2252101337.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

/tmp/ipykernel_632180/2252101337.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK['cell_id'] = one_fov_noLK['cell_id'].astype(int)
/tmp/ipykernel_632180/2252101337.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK_dists['group'].loc[one_fov_noLK_dists['minLKdist_um'] >= 30] = '30+ um'
/tmp/ipykernel_632180/2252101337.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

/tmp/ipykernel_632180/2252101337.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK['cell_id'] = one_fov_noLK['cell_id'].astype(int)
/tmp/ipykernel_632180/2252101337.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK_dists['group'].loc[one_fov_noLK_dists['minLKdist_um'] >= 30] = '30+ um'
/tmp/ipykernel_632180/2252101337.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

/tmp/ipykernel_632180/2252101337.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK['cell_id'] = one_fov_noLK['cell_id'].astype(int)
/tmp/ipykernel_632180/2252101337.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK_dists['group'].loc[one_fov_noLK_dists['minLKdist_um'] >= 30] = '30+ um'
/tmp/ipykernel_632180/2252101337.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

...

/tmp/ipykernel_632180/2252101337.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK['cell_id'] = one_fov_noLK['cell_id'].astype(int)
/tmp/ipykernel_632180/2252101337.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK_dists['group'].loc[one_fov_noLK_dists['minLKdist_um'] >= 30] = '30+ um'
/tmp/ipykernel_632180/2252101337.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

/tmp/ipykernel_632180/2252101337.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK['cell_id'] = one_fov_noLK['cell_id'].astype(int)
/tmp/ipykernel_632180/2252101337.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK_dists['group'].loc[one_fov_noLK_dists['minLKdist_um'] >= 30] = '30+ um'
/tmp/ipykernel_632180/2252101337.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

/tmp/ipykernel_632180/2252101337.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK['cell_id'] = one_fov_noLK['cell_id'].astype(int)
/tmp/ipykernel_632180/2252101337.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK_dists['group'].loc[one_fov_noLK_dists['minLKdist_um'] >= 30] = '30+ um'
/tmp/ipykernel_632180/2252101337.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

/tmp/ipykernel_632180/2252101337.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK['cell_id'] = one_fov_noLK['cell_id'].astype(int)
/tmp/ipykernel_632180/2252101337.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK_dists['group'].loc[one_fov_noLK_dists['minLKdist_um'] >= 30] = '30+ um'
/tmp/ipykernel_632180/2252101337.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

/tmp/ipykernel_632180/2252101337.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK['cell_id'] = one_fov_noLK['cell_id'].astype(int)
/tmp/ipykernel_632180/2252101337.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov_noLK_dists['group'].loc[one_fov_noLK_dists['minLKdist_um'] >= 30] = '30+ um'
/tmp/ipykernel_632180/2252101337.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

In [15]:
# concatentate DFs from all FOVs
all_fovs_noLK_dists = pd.concat(All_DFs)
all_fovs_noLK_dists

,patients,celltype_detail,FOV,cell_id,Timepoint,minLKdist_um,group
19937,p1,DC,09,1,B,98.3466,30+ um
19938,p1,MonocytesCD14,09,2,B,91.8216,30+ um
19939,p1,DC,09,3,B,80.0910,30+ um
19940,p1,MonocytesCD16,09,4,B,74.8404,30+ um
19941,p1,MonocytesCD14,09,5,B,26.7534,
...,...,...,...,...,...,...,...
500712,p6,MonocytesCD16,01,12006,A,111.7134,30+ um
500713,p6,MonocytesProgenitor,01,12008,A,71.5716,30+ um
500714,p6,ErythroidProgenitors,01,12010,A,81.2844,30+ um
500715,p6,MonocytesProgenitor,01,12013,A,106.6608,30+ um


In [16]:
all_fovs_noLK_dists.to_csv('Ligand_receptor_input.csv')